#Importando bibliotecas

In [ ]:
import pandas as pd
import sqlite3
import io

#Abertura e formatação dos arquivos


##Tabela PANdados

In [191]:
pan_csv = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapandados.csv', encoding='ISO-8859-1', sep=';')

###Planos de Ação Nacional

In [189]:
df = pan_csv

df = df.iloc[:, [0,1,2,3,8,15,16]]

df = df.rename(
    columns={
        'idPan': 'IDPan',
        'panNome': 'Nome_Pan',
        'panNomeCompleto': 'Nome_Completo',
        'panNomeFantasia': 'Nome_Fantasia',
        'panCentro': 'Centro',
        'panPortaria': 'Portaria',
        'panProcesso': 'Processo'
    }
)

#NOMEPAN PARECE INÚTIL, PELOS SEGUINTES CRITÉRIOS:
# O nome está contido em 'Nome_Fantasia'
# O ciclo está contido em ciclo
# (Finalizado) (Elaborado) está contido em status
df['Nome_Pan'] = df['Nome_Pan'].str.replace('\([A-Za-z]+\)','',regex=True)

df_planos = df
df_planos

,IDPan,Nome_Pan,Nome_Completo,Nome_Fantasia,Centro,Portaria,Processo
0,CBC_01.01,Lepidópteros,Plano de Ação Nacional para a Conservação de L...,Lepidópteros,CBC,Portaria 92 (27/08/2010),02070.002646/2010-11
1,CBC_02.01,Insetos Polinizadores,Plano de Ação Nacional para a Conservação Plan...,Insetos Polinizadores,CBC,Portaira 1145 (05/12/2022),02176.000061/2019-17
2,CECAV_01.01,Morceguinho-do-cerrado,Plano de Ação Nacional para Conservação do Mor...,Morceguinho-do-cerrado,CECAV,Portaria 89 (27/08/2010),02070.003685/2009-94
3,CECAV_02.01,Cavernas do Rio São Francisco,Plano de Ação Nacional para a Conservação do P...,Cavernas do Rio São Francisco,CECAV,Portaria 18 (17/02/2012),02070.002654/2010-50
4,CECAV_03.01,Cavernas do Brasil,Plano de Ação Nacional para a Conservação do P...,Cavernas do Brasil,CECAV,Portaria 646 (09/08/2022),02667.000076/2019-45
...,...,...,...,...,...,...,...
100,RAN_10.02,Herpetofauna do Nordeste - 2º ciclo,Plano de Ação Nacional para a Conservação da H...,Herpetofauna do Nordeste,RAN,Portaria 354 (25/07/2019),02071.000134/2018-50
101,RAN_11.02,Herpetofauna da MA do Sudeste - 2º ciclo,Plano de Ação Nacional para Conservação da Her...,Herpetofauna da MA do Sudeste,RAN,NaN,NaN
102,RAN_12.02,CERPAN - 2º ciclo,Plano de Ação Nacional para Conservação das Es...,Cerrado e Pantanal,RAN,NaN,NaN
103,TAMAR_01.01,Tartarugas Marinhas - 1º ciclo,Plano de Ação Nacional para a Conservação das ...,Tartarugas Marinhas,TAMAR,Portaria 135 (23/12/2010),NaN


###Abrangência

In [ ]:
df = pan_csv

#Apenas divido as colunas necessárias
df = df.iloc[:, [0,4,5]]
df_abrangencia = df
df_abrangencia

,idPan,panAbrangenciaTaxonomica,panAbrangenciaGeografica
0,CBC_01.01,Intraclasse,Nacional
1,CBC_02.01,Intraclasse,Nacional
2,CECAV_01.01,Monoespecífico,Bioma
3,CECAV_02.01,Multiclasse,Ecossistema
4,CECAV_03.01,Multiclasse,Ecossistema
...,...,...,...
100,RAN_10.02,Multiclasse,Bioma
101,RAN_11.02,Multiclasse,Bioma
102,RAN_12.02,Multiclasse,Bioma
103,TAMAR_01.01,Intraclasse,Bioma


###Site

In [ ]:
df = pan_csv

#Seleciono apenas as colunas necessárias
df = df.iloc[:, [0,3,17,18]]

df = df.rename(
    columns={
        "idPan": "IDPan",
        "panNomeFantasia": "Titulo",
        "panLogo": "Logo_Site",
        "panSite": "Link_Site"
    }
)

#Excluo sites sem link
df = df.dropna(subset=['Link_Site'])

df_site = df
df_site

,IDPan,Titulo,Logo_Site,Link_Site
0,CBC_01.01,Lepidópteros,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
1,CBC_02.01,Insetos Polinizadores,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
2,CECAV_01.01,Morceguinho-do-cerrado,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
3,CECAV_02.01,Cavernas do Rio São Francisco,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
4,CECAV_03.01,Cavernas do Brasil,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
...,...,...,...,...
100,RAN_10.02,Herpetofauna do Nordeste,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
101,RAN_11.02,Herpetofauna da MA do Sudeste,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
102,RAN_12.02,Cerrado e Pantanal,https://www.gov.br/icmbio/pt-br/assuntos/biodi...,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
103,TAMAR_01.01,Tartarugas Marinhas,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...


###Status

In [174]:
df = pan_csv

#Seleciono as colunas que desejo (A cópia evita um erros)
df = df.iloc[:, [0,6,7,9,13]].copy()

df = df.rename(
    columns={
        'panInicioData': 'Data_Inicio',
        'panFimData': 'Data_Fim',
        'panCiclo': 'Num_Ciclo',
        'panStatus': 'Status',
        'idPan': 'IDPan'
    }
)

#Converto as datas de fim para datetime
df[df.columns[4]] = pd.to_datetime(df['Data_Fim'],dayfirst=True)
df['Data_Fim'] = df['Data_Fim'].dt.date

#Converto as datas de inicio para datetime
#(errors='coerce') transforma informações que não são datas para null
df[df.columns[3]] = pd.to_datetime(df['Data_Inicio'],dayfirst=True, errors='coerce')
df['Data_Inicio'] = df['Data_Inicio'].dt.date

#Substitui a string "xº ciclo" por apenas seu número
df[df.columns[1]] = df['Num_Ciclo'].str.extract('(\d+)')

df_status = df
df_status


,IDPan,Num_Ciclo,Status,Data_Inicio,Data_Fim
0,CBC_01.01,1,Finalizado,2010-08-27,2015-12-01
1,CBC_02.01,1,Em execução,2022-12-05,2027-12-31
2,CECAV_01.01,1,Finalizado,2010-08-27,2015-09-01
3,CECAV_02.01,1,Finalizado,2012-02-17,2017-02-01
4,CECAV_03.01,1,Em execução,2022-08-09,2027-08-01
...,...,...,...,...,...
100,RAN_10.02,2,Em execução,2018-12-28,2024-01-01
101,RAN_11.02,2,Previsto,NaT,NaT
102,RAN_12.02,2,Previsto,NaT,NaT
103,TAMAR_01.01,1,Finalizado,2010-12-23,2015-12-01


##Tabela PANespecies

In [ ]:
#Abro o csv e gero um dataframe com suas informações
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanespecies.csv', encoding='unicode_escape', sep=';')

#Renomeio alguma colunas
df = df.rename(
    columns={
        "idTaxon": "IDEspecie",
        "taxonNome": "Nome_Cientifico",
        "idPan": "IDPan"
    }
)
df

#Removo a coluna 315, que tinha uma espécie apagada
df = df[df.Nome_Cientifico != 'não é mais subespécie']

#Removo a coluna pan_nome, que é redundante
df = df.drop(columns=['pan_nome'])

#Corrijo os nomes de 534 e 856
df = df.replace({'Crypturellus noctivagus noctivagus Crypturellus noctivagus':'Crypturellus noctivagus noctivagus'})

#Quebro a coluna nome científico em 3 nomes
df[['Nome_Genero', 'Epiteto_Especifico', 'Subespecie']] = df['Nome_Cientifico'].str.split(expand=True)

#Mudo a ordem das colunas
df = df.iloc[:, [2,0,1,3,4,5]]

#Possíveis mudanças, excluir IDEspecie, que parece inútil, excluir nome_cientifico

#Salvo o dataframe como "df_especies"
df_especies = df
df_especies

#Instâncias com problemas:
# 315, "não é mais subespécie" : nome apagado
# 534 e 856, "Crypturellus noctivagus noctivagus Crypturellus noctivagus" : nome repetido

##Tabela PANbiomas

In [205]:
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanbiomas.csv', encoding='ISO-8859-1', sep=';')

df = df.iloc[:, [0,2]]

df = df.rename(
    columns={
        'idPan': 'IDPan',
        'panBioma': 'Bioma'
    }
)

df_biomas = df
df_biomas

,IDPan,Bioma
0,CBC_01.01,Amazônia
1,CBC_01.01,Caatinga
2,CBC_01.01,Cerrado
3,CBC_01.01,Mata Atlântica
4,CBC_01.01,Pampa
...,...,...
221,RAN_09.02,Pampa
222,RAN_10.02,Mata Atlântica
223,RAN_10.02,Caatinga
224,TAMAR_01.01,Marinho


##Tabela PANestados

In [203]:
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanestados.csv', encoding='ISO-8859-1', sep=';')

df = df.iloc[:, [0,2]]

df = df.rename(
    columns={
        'idPan': 'IDPan',
        'siglaEstado': 'Estado'
    }
)

#Crio um dicionário
regioes = {
    'AC' : 'Norte',
    'AL' : 'Nordeste',
    'AP' : 'Norte',
    'AM' : 'Norte',
    'BA' : 'Nordeste',
    'CE' : 'Nordeste',
    'DF' : 'Centro-Oeste',
    'ES' : 'Sudeste',
    'GO' : 'Centro-Oeste',
    'MA' : 'Nordeste',
    'MT' : 'Centro-Oeste',
    'MS' : 'Centro-Oeste',
    'MG' : 'Sudeste',
    'PA' : 'Norte',
    'PB' : 'Nordeste',
    'PR' : 'Sul',
    'PE' : 'Nordeste',
    'PI' : 'Nordeste',
    'RJ' : 'Sudeste',
    'RN' : 'Nordeste',
    'RS' : 'Sul',
    'RO' : 'Norte',
    'RR' : 'Norte',
    'SC' : 'Sul',
    'SP' : 'Sudeste',
    'SE' : 'Nordeste',
    'TO' : 'Norte'
}

#Adiciono nova coluna a partir do dicionário
df['Regiao'] = df['Estado'].map(regioes)

df_estados = df
df_estados

,IDPan,Estado,Regiao
0,CEMAVE_29.02,MG,Sudeste
1,CEMAVE_29.02,TO,Norte
2,CEMAVE_29.02,GO,Centro-Oeste
3,CEMAVE_26.03,PR,Sul
4,CEMAVE_26.03,RJ,Sudeste
...,...,...,...
515,CECAV_03.01,RR,Norte
516,CECAV_03.01,SC,Sul
517,CECAV_03.01,SP,Sudeste
518,CECAV_03.01,SE,Nordeste


#Colocando dados no sql e criando .dump

In [175]:
#Crio a database e a conexão
conn = sqlite3.connect('test_database')
c = conn.cursor()

#Crio o esquema para a tabela
c.execute('CREATE TABLE IF NOT EXISTS especies (id_expecie number, nome_cientifico text, idPan text, pan_nome text)')
conn.commit()

#Passo as informações do dataframe pandas para a tabela sql
df_especies.to_sql('especies', conn, if_exists='replace', index = False)

#Crio o dump em sql
with open('pandados.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)
conn.close()